## Part 1

In [6]:
import numpy as np

import torch
from torch.autograd import Variable

sizes = [1, 2, 1]

x = 10*torch.rand(sizes[0]).double()

w1 = Variable(10*torch.randn(sizes[1], sizes[0]).double(), requires_grad=True)
w2 = Variable(10*torch.randn(sizes[2], sizes[1]).double(), requires_grad=True)
b1 = Variable(10*torch.randn(sizes[1], 1).double(), requires_grad=True)
b2 = Variable(10*torch.randn(sizes[2], 1).double(), requires_grad=True)

print(w1)
print(x)

f = torch.sigmoid(w2.dot(torch.sigmoid(w1.dot(x) + b1)) + b2)

tensor([[-14.9485],
        [  8.1997]], dtype=torch.float64, requires_grad=True)
tensor([6.0002], dtype=torch.float64)


RuntimeError: dot: Expected 1-D argument self, but got 2-D